In [ ]:
# Import packages
import TS_Custom
from TS_Custom import *


#initialize globalvariables
buystrategy,sellstrategy,buylong,buyshort,buycall,buyput = get_Trade_Signals_fromDB()
    
def getStrategyResult(symbol):
    #for symbol in symbol_list :
    try:
        #print (datetime.now().strftime('%H:%M'))            
        # get price history from tradestation
        mins = 1
        
        #bars are set to 600 so that ATR for UTBot Buy is accurate     
        barsBack=int(barssince7am(mins))
        if (barsBack <= 80):
            barsBack = 80
        
        df = getPriceHistory_TS(symbol,str(barsBack), str(mins))
        df = getHA(df)
       
        # get options data from td ameritrade
        content = getOptionChainDatafromTD(symbol)
        # retrieve call symbol
        call_optionSymbol = getCallOptionSymbol(content)

        # retrieve put symbol
        put_optionSymbol = getPutOptionSymbol(content) 

     
        
        df = Supertrend(df, 10, 1)
        #df = UTBot_Sell(df,2,1,False)
        #df = UTBot_Buy(df,2,300,False)
                
        
        atr = df['ATR']
        ema9 = df['ema9']
        ema21 = df['ema21']
        
        #retrieve keys     
        last_key = atr.keys()[-1]
        previous_key = atr.keys()[-2]
        previous_key_2 = atr.keys()[-3]
        
        # Strategy Code
        #variables
        datetime_now = datetime.now().strftime('%Y-%m-%d %H-%M-%S')
        strategy = 'BUY_SUPERTREND'

        # Buy Stock  
        if (buylong == True):
            if (df['ST_Lowerband'][previous_key]> 0) and (df['ST_Upperband'][previous_key_2]> 0) :

                if (not getSymbolStockPositionsfromDB(symbol)) :                        
                    
                    print('symbol = ', symbol)
                    # get quote from tradestation
                    quote = getQuote(symbol)
                    #midPrice = round((quote['Bid'] + quote['Ask'])/2, 2)
                    bidPrice = round((quote['Bid']), 2)
                    askPrice = round((quote['Ask']), 2)
                    midPrice = round((askPrice + bidPrice)/2,2)
                    #openPrice = float(round((df['Open'][last_key]), 2))
                    limitprice = float(round(askPrice+0.01,2))
                    #limitprice = midPrice

                    equity_bidaskpercent = getEquity_BidAskPercent(symbol,bidPrice,askPrice)
                    #print("equity_bidaskpercent:",equity_bidaskpercent, 'for', symbol)   

                    if (equity_bidaskpercent <= 0.4) :
                        quantity=int(4000/(limitprice))

                        print("ALL CONDITIONS FOR BUY STOCK MET FOR ", symbol)

                        totalCost = round((limitprice*quantity),2)
                        openpl = round((quantity * (bidPrice-limitprice)), 2)
                        openpl_percent = round(((openpl/totalCost)*100), 2)
                        mktvalue = mktvalue = round((quantity * bidPrice),2)
                        triggerprice = round(quantity * 0.40 * atr[last_key],2)
                        previous_openpl = round((quantity * (bidPrice-limitprice)), 2)
                        
                        mins_5 = 5
                        barsback_5min = 20
                        df_5min = getPriceHistory_TS(symbol,str(barsback_5min), str(mins_5))
                        ema9_5min = df_5min['ema9']

                        #retrieve keys     
                        last_key_5min = ema9_5min.keys()[-1]
                        previous_key_5min = ema9_5min.keys()[-2]
                        
                        profit_target = float(round(limitprice + ((limitprice - df_5min['Low'][previous_key_5min]) * 2),2))
                        stop_loss = float(round(df_5min['Low'][previous_key_5min],2))
                        
                        #buy Equity in TS
                        #buyEQUITY(symbol, quantity, limitprice, tradelog_filename)

                        winsound.Beep(4000, 500)

                        # add to DB
                        addOrderOutputtoDB(datetime_now, str(symbol), 'BUY_STOCK', limitprice, quantity)
                        addPositionOutputtoDB(datetime_now, str(symbol), 'Long', quantity, limitprice,totalCost, openpl,openpl_percent,bidPrice,askPrice,mktvalue, triggerprice,previous_openpl,strategy,profit_target,stop_loss)

                        print(df[['SYMBOL','TimeStamp','ST_Upperband','ST_Lowerband', 'Close']].tail(5))
                        #df.to_csv(buystochasticresults_filename, mode = 'a', index=False)

                        #add log to DB
                        addOrderLogOutputtoDB(datetime_now, symbol, strategy,'BUY_STOCK', limitprice, quantity, '', bidPrice, askPrice,df)




        # Sell Short Stock   
        if (buyshort == True):
            if (df['ST_Upperband'][previous_key]> 0) and (df['ST_Lowerband'][previous_key_2]> 0) :

                if (not getSymbolStockPositionsfromDB(symbol)) :

                    # get quote from tradestation
                    quote = getQuote(symbol)
                    #midPrice = round((quote['Bid'] + quote['Ask'])/2, 2)
                    askPrice = round((quote['Ask']), 2)
                    bidPrice = round((quote['Bid']), 2)
                    midPrice = round((askPrice + bidPrice)/2,2)
                    #openPrice = float(round((df['Open'][last_key]), 2))
                    #limitprice = bidPrice
                    limitprice = float(round(bidPrice-0.01,2))
                    #limitprice = midPrice


                    equity_bidaskpercent = getEquity_BidAskPercent(symbol,bidPrice,askPrice)
                    #print("equity_bidaskpercent:",equity_bidaskpercent, 'for', symbol)

                    if (equity_bidaskpercent <= 0.4) :

                        quantity=int(4000/(limitprice))

                        print("ALL CONDITIONS FOR SELLSHORT MET FOR ", symbol)

                        totalCost = round((limitprice*quantity),2)
                        openpl = round((quantity * (limitprice-askPrice)), 2)
                        openpl_percent = round(((openpl/totalCost)*100), 2)
                        mktvalue = round((quantity * bidPrice),2)
                        triggerprice = round(quantity * 0.40 * atr[last_key],2)
                        previous_openpl = round((quantity * (limitprice-askPrice)), 2)
                        
                        mins_5 = 5
                        barsback_5min = 20
                        df_5min = getPriceHistory_TS(symbol,str(barsback_5min), str(mins_5))
                        ema9_5min = df_5min['ema9']

                        #retrieve keys     
                        last_key_5min = ema9_5min.keys()[-1]
                        previous_key_5min = ema9_5min.keys()[-2]

                        
                        profit_target = float(round(limitprice - ((df_5min['High'][previous_key_5min] - limitprice )* 2),2))
                        stop_loss = float(round(df_5min['High'][previous_key_5min],2))
                        

                        #sellshort Equity in TS
                        #sellshortEQUITY(symbol, quantity, limitprice, tradelog_filename)

                        winsound.Beep(4000, 500)
                        
                        #add to DB
                        addOrderOutputtoDB(datetime_now, str(symbol), 'SELLSHORT_STOCK', limitprice, quantity)
                        addPositionOutputtoDB(datetime_now, str(symbol), 'Short', quantity, limitprice, totalCost, openpl,openpl_percent,bidPrice,askPrice,mktvalue, triggerprice,previous_openpl,strategy,profit_target,stop_loss)

                        print(df[['SYMBOL','TimeStamp','ST_Upperband','ST_Lowerband', 'Close']].tail(5))
                        #df.to_csv(buystochasticresults_filename, mode = 'a', index=False)

                        #add log to DB
                        addOrderLogOutputtoDB(datetime_now, symbol, strategy,'SELLSHORT_STOCK', limitprice, quantity, '', bidPrice, askPrice,df)


        
        
        '''
        # Buy Options
        if (buycall == True):

            # Call Option
            if ((df['ema9_angle'][previous_key] >= 25) and \
                (df['ema9_angle'][previous_key] > df['ema9_angle'][previous_key_2]) and \
                (df['linear_reg_slope'][previous_key] > df['linear_reg_slope'][previous_key_2]) and \
                (df['HA_Close'][previous_key] > df['HA_Open'][previous_key]) and \
                #(df['HA_Close'][last_key] > df['HA_Open'][last_key]) and \
                (rsi[previous_key] <= 70) and \
                (ema9[previous_key] > ema21[previous_key]) and \
                (ema21[previous_key] > ema50[previous_key])) :

                if (((df['linear_reg_slope'][previous_key] > 0) and \
                    (df['linear_reg_slope'][previous_key_2] < 0)) or \
                    ((df['linear_reg_slope'][previous_key] > 0) and \
                     (df['linear_reg_slope'][previous_key_2] > 0) and \
                     (df['linear_reg_slope'][previous_key_3] < 0))) : 


                    #if (not getSymbolCallPositions(acct,symbol)) and (not getSymbolCallOrders_Received(acct,symbol)) :
                    if (not getSymbolCallPositionsfromDB(symbol)) :

                        bidaskpercent = getBidAskPercent(content, "callExpDateMap")
                        print("bidaskpercent:",bidaskpercent,'% for', symbol)
                        if bidaskpercent <= 5 :
                            askPrice = getAskPrice(content, "callExpDateMap")
                            bidPrice = getBidPrice(content, "callExpDateMap")
                            limitprice = askPrice
                            print('limit',limitprice)

                            if (symbol == "TSLA") or (symbol == "GOOG") :
                                quantity = 2
                            else :
                                quantity=int(15000/(limitprice*100))

                            print("ALL CONDITIONS FOR BUY CALL OPTION MET FOR ", symbol)

                            totalCost = round((limitprice*quantity*100),2)
                            openpl = round((quantity * 100* (bidPrice-limitprice)), 2)
                            openpl_percent = round(((openpl/totalCost)*100), 2)
                            mktvalue = mktvalue = round((quantity * 100 * bidPrice),2)
                            #triggerprice = round(limitprice + atr[last_key],2)
                            triggerprice = round(quantity * atr[last_key],2)
                            previous_openpl = round((quantity * 100 * (bidPrice-limitprice)), 2)
                            profit_target = limitprice * 1.0065
                            stop_loss = limitprice * 0.995

                            winsound.Beep(4000, 500)

                            #Buy Call Options in TS
                            buyCALL(symbol, quantity, df, call_optionSymbol, tradelog_filename, content)


                            # add to DB
                            addOrderOutputtoDB(datetime_now, str(call_optionSymbol), 'BUY_CALL', limitprice, quantity)
                            addPositionOutputtoDB(datetime_now, str(call_optionSymbol), 'Long', quantity, limitprice,totalCost, openpl, openpl_percent, bidPrice,askPrice,mktvalue, triggerprice,previous_openpl,strategy,profit_target,stop_loss)


                            #add log to DB
                            addOrderLogOutputtoDB(datetime_now, str(call_optionSymbol), strategy,'BUY_CALL', limitprice, quantity, '', bidPrice, askPrice,df)



        # Put Option

        if (buyput == True):

            if ((df['ema9_angle'][previous_key] < 0) and \
                (df['linear_reg_slope'][previous_key] < df['linear_reg_slope'][previous_key_2]) and \
                (df['ema9_angle'][last_key] < 0) and \
                (df['HA_Close'][previous_key] < df['HA_Open'][previous_key]) and \
                #(df['HA_Close'][last_key] < df['HA_Open'][last_key]) and \
                (rsi[previous_key] >=35 ) and \
                (ema9[previous_key] < ema21[previous_key]) and \
                (ema21[previous_key] < ema50[previous_key])) :

                if (((df['linear_reg_slope'][previous_key] < 0) and \
                    (df['linear_reg_slope'][previous_key_2] > 0)) or \
                    ((df['linear_reg_slope'][previous_key] < 0) and \
                     (df['linear_reg_slope'][previous_key_2] < 0) and \
                     (df['linear_reg_slope'][previous_key_3] > 0))) : 

                    #if (not getSymbolPutPositions(acct,symbol)) and (not getSymbolPutOrders_Received(acct,symbol)) :
                    if (not getSymbolPutPositionsfromDB(symbol)) :

                        bidaskpercent = getBidAskPercent(content, "putExpDateMap")
                        print("bidaskpercent:",bidaskpercent,'% for', symbol)
                        if bidaskpercent <= 5 :
                            askPrice = getAskPrice(content, "putExpDateMap")
                            bidPrice = getBidPrice(content, "putExpDateMap")
                            limitprice = askPrice
                            print('limit',limitprice)

                            if (symbol == "TSLA") or (symbol == "GOOG") :
                                quantity = 2
                            else :
                                quantity=int(15000/(limitprice*100))

                            print("ALL CONDITIONS FOR BUY PUT OPTION MET FOR ", symbol)

                            totalCost = round((limitprice*quantity*100),2)
                            openpl = round((quantity * 100* (bidPrice-limitprice)), 2)
                            openpl_percent = round(((openpl/totalCost)*100), 2)
                            mktvalue = mktvalue = round((quantity * 100 * bidPrice),2)
                            #triggerprice = round(limitprice + atr[last_key],2)
                            triggerprice = round(quantity * atr[last_key],2)
                            previous_openpl = round((quantity * 100 * (bidPrice-limitprice)), 2)
                            profit_target = limitprice * 1.0065
                            stop_loss = limitprice * 0.995

                            winsound.Beep(4000, 500)

                            #Buy Put Options in TS
                            buyPUT(symbol, quantity, df, put_optionSymbol, tradelog_filename, content)


                            # add to DB
                            addOrderOutputtoDB(datetime_now, str(put_optionSymbol), 'BUY_PUT', limitprice, quantity)
                            addPositionOutputtoDB(datetime_now, str(put_optionSymbol), 'Long', quantity, limitprice,totalCost, openpl, openpl_percent, bidPrice,askPrice,mktvalue, triggerprice,previous_openpl,strategy,profit_target,stop_loss)


                            #add log to DB
                            addOrderLogOutputtoDB(datetime_now, str(put_optionSymbol), strategy,'BUY_PUT', limitprice, quantity, '', bidPrice, askPrice,df)

            '''
            #print("******************")

    except Exception as e:
        print(symbol)
        traceback.print_exc()
        print(e)
        
while True:        
        
    now = datetime.now()
    today800=now.replace(hour=8,minute=0,second=0,microsecond=0)
    today1500=now.replace(hour=23,minute=0,second=0,microsecond=0)
    #if (True) :
    if (now > today800 and now < today1500 ) :
        #import Multithreading file for this function below
        #symbol_list = getMomentumStocks()
        #print(symbol_list)
        #getStrategyResult(symbol_list)

        #get stocks list from multithread YF data
        #symbol_list = getStocks()
        #print('SYMBOL LIST', symbol_list)        

        buystrategy,sellstrategy,buylong,buyshort,buycall,buyput = get_Trade_Signals_fromDB()

        if (buystrategy == True) :   
            
            
            #symbol_list1 = ['AAPL'] 
            
                      
            symbol_list1 = pd.read_csv('tickers1.csv', header=None)[0].tolist()
            print(symbol_list1)
            # Time starts from here
            start = time.time()


            with concurrent.futures.ThreadPoolExecutor(max_workers=len(symbol_list1)) as executor:
                executor.map(getStrategyResult,symbol_list1)

            # Total time calculated
            print(now, ' - symbol_list1 took ', time.time()-start, 'seconds.')

            print("**************************************************************")
            time.sleep(30)

            
            '''
            
            buystrategy,sellstrategy,buylong,buyshort,buycall,buyput = get_Trade_Signals_fromDB()
            symbol_list2 = pd.read_csv('tickers2.csv', header=None)[0].tolist()
            print(symbol_list2)
            
            
            # Time starts from here
            start = time.time()


            with concurrent.futures.ThreadPoolExecutor(max_workers=len(symbol_list2)) as executor:            
                executor.map(getStrategyResult,symbol_list2)

            # Total time calculated
            print(now, ' - symbol_list2 took ', time.time()-start, 'seconds.')

            print("**************************************************************")
            time.sleep(30)
            
            
            
            buystrategy,sellstrategy,buylong,buyshort,buycall,buyput = get_Trade_Signals_fromDB()
            symbol_list3 = pd.read_csv('tickers3.csv', header=None)[0].tolist()
            print(symbol_list3)
            
            # Time starts from here
            start = time.time()        

            with concurrent.futures.ThreadPoolExecutor(max_workers=len(symbol_list3)) as executor:
                #print(threading.currentThread().ident)
                executor.map(getStrategyResult,symbol_list3)

            # Total time calculated
            print(now, ' - symbol_list3 took ', time.time()-start, 'seconds.')

            print("**************************************************************")
            time.sleep(30)
            
                       
            
            
            buystrategy,sellstrategy,buylong,buyshort,buycall,buyput = get_Trade_Signals_fromDB()  
            #symbol_list0=getMomentumStocks_TS_Scanner().head(10)['Symbol'].tolist()
            momentum_df=getMomentumStocks_TS_Scanner()
            if (not momentum_df.empty):
                symbol_list0=momentum_df.head(10)['Symbol'].tolist()
                print("Momentum Scanner Stocks:")
                print(symbol_list0)
                # Time starts from here
                start = time.time()
                if(len(symbol_list0)>0):
                    with concurrent.futures.ThreadPoolExecutor(max_workers=len(symbol_list0)) as executor:
                        executor.map(getStrategyResult,symbol_list0)

                # Total time calculated
                print(now, ' - symbol_list0 took ', time.time()-start, 'seconds.')

                print("**************************************************************")
                time.sleep(30)

          '''  
        
    
   




['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 09:08:53.207579  - symbol_list1 took  1.3056204319000244 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 09:09:24.571501  - symbol_list1 took  1.2620162963867188 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
Last Date was  02-02-2023
Date was MISMATCH. Had to reset BARSBACK for  CCL
Dataframe is ACCURATE after resetting barsBack
2023-02-03 09:09:55.868012  - symbol_list1 took  2.4963462352752686 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
symbol =  TSLA
ALL CONDITIONS FOR BUY STOCK MET FOR  TSLA
Last Date was  02-02-20

['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
symbol =  AMD
ALL CONDITIONS FOR BUY STOCK MET FOR  AMD
    SYMBOL               TimeStamp  ST_Upperband  ST_Lowerband  Close
130    AMD  02-03-2023 09:13:00 AM     87.632276           NaN  87.62
131    AMD  02-03-2023 09:14:00 AM     87.632276           NaN  87.61
132    AMD  02-03-2023 09:15:00 AM     87.632276           NaN  87.67
133    AMD  02-03-2023 09:16:00 AM           NaN     87.304915  87.62
134    AMD  02-03-2023 09:17:00 AM           NaN     87.304915  87.59
2023-02-03 09:15:59.449752  - symbol_list1 took  2.5131685733795166 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 09:16:32.002827  - symbol_list1 took  4.749173402786255 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'C

['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 09:29:39.777948  - symbol_list1 took  1.5061695575714111 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
Last Date was  02-02-2023
Date was MISMATCH. Had to reset BARSBACK for  CCL
ALL CONDITIONS FOR SELLSHORT MET FOR  AMZN
Dataframe is ACCURATE after resetting barsBack
ALL CONDITIONS FOR SELLSHORT MET FOR  NVDA
    SYMBOL               TimeStamp  ST_Upperband  ST_Lowerband   Close
145   AMZN  02-03-2023 09:27:00 AM           NaN    107.518333  108.00
146   AMZN  02-03-2023 09:28:00 AM           NaN    107.518333  107.86
147   AMZN  02-03-2023 09:29:00 AM           NaN    107.518333  107.45
148   AMZN  02-03-2023 09:30:00 AM    107.859731           NaN  107.08
149   AMZN  02-03-2023 09:31:00 AM    107.796943           NaN  107.28
    SYMBOL               TimeStamp  S

Traceback (most recent call last):
  File "C:\Users\VivekP\AppData\Local\Temp\ipykernel_8896\1753779578.py", line 34, in getStrategyResult
    df = Supertrend(df, 10, 1)
  File "C:\Test_AlphaOne\TS_Custom.py", line 3688, in Supertrend
    high = df['HA_High']
TypeError: list indices must be integers or slices, not str
Traceback (most recent call last):
  File "C:\Users\VivekP\AppData\Local\Temp\ipykernel_8896\1753779578.py", line 34, in getStrategyResult
    df = Supertrend(df, 10, 1)
  File "C:\Test_AlphaOne\TS_Custom.py", line 3688, in Supertrend
    high = df['HA_High']
TypeError: list indices must be integers or slices, not str


['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 09:40:02.816531  - symbol_list1 took  1.3193721771240234 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 09:40:34.183938  - symbol_list1 took  1.2202608585357666 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 09:41:05.449120  - symbol_list1 took  1.2304785251617432 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
Last Date was  02-02-2023
Date was MISMATCH. Had to reset BARSBACK for  SPY
Dataframe is ACCURATE after resetting barsBack
2023-02-03 09:41:36.729149  - symbol_list1 took  2.211867332458496 seconds.
*****

...

['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 10:22:29.848390  - symbol_list1 took  1.4351232051849365 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 10:23:01.351354  - symbol_list1 took  1.371946096420288 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
Last Date was  02-02-2023
Date was MISMATCH. Had to reset BARSBACK for  LABU
Dataframe is ACCURATE after resetting barsBack
2023-02-03 10:23:32.771395  - symbol_list1 took  2.4706640243530273 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 10:24:05.290464  - symbol_list1 took  3.5131688117980957 seconds.
****

['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 10:39:01.498167  - symbol_list1 took  1.3771417140960693 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 10:39:32.924092  - symbol_list1 took  1.432755470275879 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
ALL CONDITIONS FOR SELLSHORT MET FOR  SPY
ALL CONDITIONS FOR SELLSHORT MET FOR  AAPL
ALL CONDITIONS FOR SELLSHORT MET FOR  NFLX
    SYMBOL               TimeStamp  ST_Upperband  ST_Lowerband   Close
215    SPY  02-03-2023 10:37:00 AM           NaN    416.564626  416.81
216    SPY  02-03-2023 10:38:00 AM           NaN    416.572164  416.87
217    SPY  02-03-2023 10:39:00 AM           NaN    416.572164  416.34
218    SPY  02-03-2023 10:4

2023-02-03 10:53:17.696120  - symbol_list1 took  2.1546809673309326 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 10:53:49.911431  - symbol_list1 took  2.0783236026763916 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 10:54:22.056701  - symbol_list1 took  1.5437471866607666 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 10:54:53.640315  - symbol_list1 took  1.4375741481781006 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 10:55:25.148403  - symbol_list1 took  1.3725893497467

2023-02-03 11:08:40.199750  - symbol_list1 took  1.4842009544372559 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 11:09:11.735912  - symbol_list1 took  1.611022710800171 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 11:09:43.401309  - symbol_list1 took  1.4296586513519287 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 11:10:14.878262  - symbol_list1 took  1.427476167678833 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 11:10:46.368546  - symbol_list1 took  1.367467403411865

2023-02-03 11:25:04.118667  - symbol_list1 took  1.4647974967956543 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 11:25:35.665711  - symbol_list1 took  1.574366569519043 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 11:26:07.301236  - symbol_list1 took  1.4671108722686768 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 11:26:38.805881  - symbol_list1 took  1.4548594951629639 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-02-03 11:27:10.298211  - symbol_list1 took  1.48071050643920